# Comparison between "Comprehensive upper-air observation network from 1905 to present" and "Insitu IGRA radiosoundings baseline network"

**Contains modified Copernicus Climate Change Service Information 2020**
under [License](https://apps.ecmwf.int/datasets/licences/copernicus/)

*Copernicus Climate Change Service (C3S) - Upper Air Data Service (2020)*

The purpose of this IPython Notebook is to find differences in both mentioned data sets and to compare them. 

    Author: U. Voggenberger
    Date: 02.2021
    Contact: ulrich.voggenberger [at] univie.ac.at
    License: C3S, 2021

In [1]:
import pandas
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua


In [2]:
def request(rqdict, source, remove_file=True):
    
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        source,rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./download/')
    z.close()
    print('Request took: ' + str(time.time() - t0) + ' seconds')
    
    files = glob.glob('./download/*.nc')
    
    if files[0].split('/')[-1].startswith('IGRA'):
        ds = xarray.open_dataset(files[0])
        ds = ds.drop(['report_id'])
        data = ds.to_dataframe()
        for i in files[1:]:
            ds = xarray.open_dataset(i)            
            data = data.append(ds.to_dataframe())

    else:
        data=eua.CDMDataset(files[0]).to_dataframe()
        for i in files[1:]:
            da = eua.CDMDataset(i).to_dataframe()
            data = data.append(da)
            
    os.remove('download.zip')
    if remove_file:
        try:
            shutil.rmtree('./download/')
        except:
            print('Error while deleting directory')

    return data

---

In [3]:
cuon = 'insitu-comprehensive-upper-air-observation-network'
igra = 'insitu-observations-igra-baseline-network'

# short period - single station

In [4]:
t = '2000-01-01/2000-01-02'
li =  ['AUM00011035']
lc = ['11035']

### CUON

In [5]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:44:54,791 INFO Welcome to the CDS
2021-02-11 13:44:54,794 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-11 13:44:55,012 INFO Request is queued
2021-02-11 13:44:56,051 INFO Request is completed
2021-02-11 13:44:56,052 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.comprehensive_upper_air.retrieve-1613047495.842095-16973-18-eb8d342c-773b-4e7d-9ec2-43ffaab2e140.zip to download.zip (42.6K)
2021-02-11 13:44:56,179 INFO Download rate 340.4K/s


Request took: 1.587897777557373 seconds
2021-02-11 13:44:56,233 - upperair.cdm | __init__ - DEBUG - [OPEN] ./download/dest_0-20000-0-11035_air_temperature.nc


2021-02-11 13:44:56,233 DEBUG [OPEN] ./download/dest_0-20000-0-11035_air_temperature.nc


2021-02-11 13:44:56,289 - upperair.cdm | to_dataframe - INFO - Reading Profile on slice(None, None, None)


2021-02-11 13:44:56,289 INFO Reading Profile on slice(None, None, None)


2021-02-11 13:44:56,296 - upperair.cdm | to_dataframe - DEBUG - Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


2021-02-11 13:44:56,296 DEBUG Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,48.23,16.370001,0,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,10000030636
1,48.23,16.370001,0,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,10000030636
2,48.23,16.370001,0,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
3,48.23,16.370001,0,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
4,48.23,16.370001,0,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,10000030636
...,...,...,...,...,...,...,...,...,...
242,48.23,16.370001,0,94800.0,10000030641,WIEN/HOHE WARTE (11035-0),272.100006,2000-01-02 12:00:00,10000030642
243,48.23,16.370001,0,97800.0,10000030641,WIEN/HOHE WARTE (11035-0),273.600006,2000-01-02 12:00:00,10000030642
244,48.23,16.370001,0,98300.0,10000030641,WIEN/HOHE WARTE (11035-0),271.500000,2000-01-02 12:00:00,10000030642
245,48.23,16.370001,0,99100.0,10000030641,WIEN/HOHE WARTE (11035-0),271.100006,2000-01-02 12:00:00,10000030642


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,48.23,16.370001,0,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,10000030636
1,48.23,16.370001,0,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,10000030636
2,48.23,16.370001,0,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
3,48.23,16.370001,0,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
4,48.23,16.370001,0,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,10000030636
...,...,...,...,...,...,...,...,...,...
242,48.23,16.370001,0,94800.0,10000030641,WIEN/HOHE WARTE (11035-0),272.100006,2000-01-02 12:00:00,10000030642
243,48.23,16.370001,0,97800.0,10000030641,WIEN/HOHE WARTE (11035-0),273.600006,2000-01-02 12:00:00,10000030642
244,48.23,16.370001,0,98300.0,10000030641,WIEN/HOHE WARTE (11035-0),271.500000,2000-01-02 12:00:00,10000030642
245,48.23,16.370001,0,99100.0,10000030641,WIEN/HOHE WARTE (11035-0),271.100006,2000-01-02 12:00:00,10000030642


### IGRA

In [6]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:45:00,003 INFO Welcome to the CDS
2021-02-11 13:45:00,004 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-11 13:45:00,048 INFO Request is queued
2021-02-11 13:45:01,087 INFO Request is completed
2021-02-11 13:45:01,088 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.insitu_reference.retrieve_test-1613047500.2988567-16973-19-a53c2934-8cbb-47f7-8507-f49b2d0898d7.zip to download.zip (2.8K)
2021-02-11 13:45:01,128 INFO Download rate 71.4K/s


Request took: 1.168215036392212 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,7000,211.449997
1,AUM00011035,2000-01-02,16.3564,48.2486,92500,271.049988
2,AUM00011035,2000-01-02,16.3564,48.2486,70000,259.649994
3,AUM00011035,2000-01-02,16.3564,48.2486,5000,208.250000
4,AUM00011035,2000-01-02,16.3564,48.2486,3000,208.050003
...,...,...,...,...,...,...
81,AUM00011035,2000-01-02,16.3564,48.2486,92600,270.850006
82,AUM00011035,2000-01-02,16.3564,48.2486,40800,236.649994
83,AUM00011035,2000-01-02,16.3564,48.2486,6180,212.649994


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,7000,211.449997
1,AUM00011035,2000-01-02,16.3564,48.2486,92500,271.049988
2,AUM00011035,2000-01-02,16.3564,48.2486,70000,259.649994
3,AUM00011035,2000-01-02,16.3564,48.2486,5000,208.250000
4,AUM00011035,2000-01-02,16.3564,48.2486,3000,208.050003
5,AUM00011035,2000-01-02,16.3564,48.2486,2000,204.449997
6,AUM00011035,2000-01-02,16.3564,48.2486,30000,225.649994
7,AUM00011035,2000-01-02,16.3564,48.2486,85000,267.850006
8,AUM00011035,2000-01-02,16.3564,48.2486,15000,214.649994


# long period - single station

In [7]:
t = '2000-01-01/2020-01-01'
li =  ['AUM00011035']
lc = ['11035']

### CUON

In [8]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:45:08,796 INFO Welcome to the CDS
2021-02-11 13:45:08,797 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-11 13:45:08,864 INFO Request is queued
2021-02-11 13:45:09,903 INFO Request is running
2021-02-11 13:45:13,733 INFO Request is completed
2021-02-11 13:45:13,734 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.comprehensive_upper_air.retrieve-1613047512.8911428-17150-16-39e1e8fa-dc48-48f8-95b6-17574ad46841.zip to download.zip (85.4M)
2021-02-11 13:45:15,670 INFO Download rate 44.1M/s  


Request took: 7.042545557022095 seconds
2021-02-11 13:45:15,807 - upperair.cdm | __init__ - DEBUG - [OPEN] ./download/dest_0-20000-0-11035_air_temperature.nc


2021-02-11 13:45:15,807 DEBUG [OPEN] ./download/dest_0-20000-0-11035_air_temperature.nc


2021-02-11 13:45:15,815 - upperair.cdm | to_dataframe - INFO - Reading Profile on slice(None, None, None)


2021-02-11 13:45:15,815 INFO Reading Profile on slice(None, None, None)


2021-02-11 13:45:16,405 - upperair.cdm | to_dataframe - DEBUG - Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


2021-02-11 13:45:16,405 DEBUG Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,48.230000,16.370001,0,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,10000030636
1,48.230000,16.370001,0,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,10000030636
2,48.230000,16.370001,0,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
3,48.230000,16.370001,0,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
4,48.230000,16.370001,0,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,10000030636
...,...,...,...,...,...,...,...,...,...
2418888,48.248951,16.356871,0,100440.0,10000055272,WIEN/HOHE WARTE (11035-0),274.119995,2020-01-01 23:30:34,10000055272
2418889,48.248951,16.356871,0,100550.0,10000055272,WIEN/HOHE WARTE (11035-0),274.079987,2020-01-01 23:30:34,10000055272
2418890,48.248951,16.356871,0,100760.0,10000055272,WIEN/HOHE WARTE (11035-0),274.100006,2020-01-01 23:30:34,10000055272
2418891,48.248951,16.356871,0,100810.0,10000055272,WIEN/HOHE WARTE (11035-0),273.940002,2020-01-01 23:30:34,10000055272


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,48.230000,16.370001,0,590.0,10000030635,WIEN/HOHE WARTE (11035-0),211.300003,2000-01-01 00:00:00,10000030636
1,48.230000,16.370001,0,760.0,10000030635,WIEN/HOHE WARTE (11035-0),206.500000,2000-01-01 00:00:00,10000030636
2,48.230000,16.370001,0,1000.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
3,48.230000,16.370001,0,1190.0,10000030635,WIEN/HOHE WARTE (11035-0),205.899994,2000-01-01 00:00:00,10000030636
4,48.230000,16.370001,0,1320.0,10000030635,WIEN/HOHE WARTE (11035-0),202.500000,2000-01-01 00:00:00,10000030636
...,...,...,...,...,...,...,...,...,...
2418888,48.248951,16.356871,0,100440.0,10000055272,WIEN/HOHE WARTE (11035-0),274.119995,2020-01-01 23:30:34,10000055272
2418889,48.248951,16.356871,0,100550.0,10000055272,WIEN/HOHE WARTE (11035-0),274.079987,2020-01-01 23:30:34,10000055272
2418890,48.248951,16.356871,0,100760.0,10000055272,WIEN/HOHE WARTE (11035-0),274.100006,2020-01-01 23:30:34,10000055272
2418891,48.248951,16.356871,0,100810.0,10000055272,WIEN/HOHE WARTE (11035-0),273.940002,2020-01-01 23:30:34,10000055272


### IGRA

In [9]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:45:25,415 INFO Welcome to the CDS
2021-02-11 13:45:25,417 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-11 13:45:25,461 INFO Request is queued
2021-02-11 13:45:26,514 INFO Request is running
2021-02-11 13:51:44,016 INFO Request is completed
2021-02-11 13:51:44,018 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.insitu_reference.retrieve_test-1613047847.5307493-17150-17-acf1a326-576e-469e-9576-eed4234aad06.zip to download.zip (11.3M)
2021-02-11 13:51:44,494 INFO Download rate 23.8M/s  


Request took: 379.40238976478577 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,43200.0,239.250000
1,AUM00011035,2000-01-02,16.3564,48.2486,7000.0,211.449997
2,AUM00011035,2000-01-02,16.3564,48.2486,92500.0,271.049988
3,AUM00011035,2000-01-02,16.3564,48.2486,70000.0,259.649994
4,AUM00011035,2000-01-02,16.3564,48.2486,5000.0,208.250000
...,...,...,...,...,...,...
1392186,AUM00011035,2019-10-01,16.3564,48.2486,61700.0,270.850006
1392187,AUM00011035,2019-10-01,16.3564,48.2486,36900.0,246.449997
1392188,AUM00011035,2019-10-01,16.3564,48.2486,43600.0,255.050003


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,2000-01-02,16.3564,48.2486,43200.0,239.250000
1,AUM00011035,2000-01-02,16.3564,48.2486,7000.0,211.449997
2,AUM00011035,2000-01-02,16.3564,48.2486,92500.0,271.049988
3,AUM00011035,2000-01-02,16.3564,48.2486,70000.0,259.649994
4,AUM00011035,2000-01-02,16.3564,48.2486,5000.0,208.250000
...,...,...,...,...,...,...
1392186,AUM00011035,2019-10-01,16.3564,48.2486,61700.0,270.850006
1392187,AUM00011035,2019-10-01,16.3564,48.2486,36900.0,246.449997
1392188,AUM00011035,2019-10-01,16.3564,48.2486,43600.0,255.050003


# short period - multiple stations

In [10]:
t = '2000-01-01/2000-02-01'
li =  ['AUM00011035', 'UKM00003693', 'GMM00010238', 'ITM00016245']
lc = ['11035', '03693', '10238', '16245']

### CUON

In [11]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:51:45,851 INFO Welcome to the CDS
2021-02-11 13:51:45,852 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-11 13:51:46,044 INFO Request is queued
2021-02-11 13:51:47,083 INFO Request is completed
2021-02-11 13:51:47,084 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.comprehensive_upper_air.retrieve-1613047906.962197-21467-12-1c5af889-64e6-4f8f-9a01-5ffcc83fc3ae.zip to download.zip (504.2K)
2021-02-11 13:51:47,264 INFO Download rate 2.8M/s 


Request took: 1.4605309963226318 seconds
2021-02-11 13:51:47,279 - upperair.cdm | __init__ - DEBUG - [OPEN] ./download/dest_0-20000-0-10238_air_temperature.nc


2021-02-11 13:51:47,279 DEBUG [OPEN] ./download/dest_0-20000-0-10238_air_temperature.nc


2021-02-11 13:51:47,294 - upperair.cdm | to_dataframe - INFO - Reading Profile on slice(None, None, None)


2021-02-11 13:51:47,294 INFO Reading Profile on slice(None, None, None)


...

2021-02-11 13:51:47,353 INFO Reading Profile on slice(None, None, None)


2021-02-11 13:51:47,356 - upperair.cdm | to_dataframe - DEBUG - Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


2021-02-11 13:51:47,356 DEBUG Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,52.82,9.930000,0,1420.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
1,52.82,9.930000,0,1680.0,10000014384,BERGEN,203.300003,2000-01-01 05:00:00,10000014384
2,52.82,9.930000,0,1850.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
3,52.82,9.930000,0,2000.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
4,52.82,9.930000,0,2430.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
...,...,...,...,...,...,...,...,...,...
3968,48.23,16.370001,0,90900.0,10000030762,WIEN/HOHE WARTE (11035-0),281.799988,2000-02-01 12:00:00,10000030763
3969,48.23,16.370001,0,91400.0,10000030762,WIEN/HOHE WARTE (11035-0),281.600006,2000-02-01 12:00:00,10000030763
3970,48.23,16.370001,0,92500.0,10000030762,WIEN/HOHE WARTE (11035-0),281.799988,2000-02-01 12:00:00,10000030763
3971,48.23,16.370001,0,95700.0,10000030762,WIEN/HOHE WARTE (11035-0),282.799988,2000-02-01 12:00:00,10000030763


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,52.82,9.930000,0,1420.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
1,52.82,9.930000,0,1680.0,10000014384,BERGEN,203.300003,2000-01-01 05:00:00,10000014384
2,52.82,9.930000,0,1850.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
3,52.82,9.930000,0,2000.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
4,52.82,9.930000,0,2430.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
...,...,...,...,...,...,...,...,...,...
3968,48.23,16.370001,0,90900.0,10000030762,WIEN/HOHE WARTE (11035-0),281.799988,2000-02-01 12:00:00,10000030763
3969,48.23,16.370001,0,91400.0,10000030762,WIEN/HOHE WARTE (11035-0),281.600006,2000-02-01 12:00:00,10000030763
3970,48.23,16.370001,0,92500.0,10000030762,WIEN/HOHE WARTE (11035-0),281.799988,2000-02-01 12:00:00,10000030763
3971,48.23,16.370001,0,95700.0,10000030762,WIEN/HOHE WARTE (11035-0),282.799988,2000-02-01 12:00:00,10000030763


### IGRA

In [12]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
}, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:51:47,504 INFO Welcome to the CDS
2021-02-11 13:51:47,505 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-11 13:51:47,575 INFO Request is queued
2021-02-11 13:51:48,614 INFO Request is running
2021-02-11 13:51:52,442 INFO Request is completed
2021-02-11 13:51:52,443 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.insitu_reference.retrieve_test-1613047910.4149601-21467-13-6bd66e69-dda6-4f1b-b9df-928d7c831d54.zip to download.zip (178.5K)
2021-02-11 13:51:52,566 INFO Download rate 1.4M/s


Request took: 5.111697196960449 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,29400.0,NaN
1,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
2,ITM00016245,2000-01-02,12.4508,41.669998,20000.0,211.649994
3,ITM00016245,2000-01-02,12.4508,41.669998,15000.0,214.850006
4,ITM00016245,2000-01-02,12.4508,41.669998,5000.0,212.649994
...,...,...,...,...,...,...
19705,AUM00011035,2000-02-01,16.3564,48.248600,2380.0,205.050003
19706,AUM00011035,2000-02-01,16.3564,48.248600,68500.0,267.450012
19707,AUM00011035,2000-02-01,16.3564,48.248600,13700.0,209.449997


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
1,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
2,ITM00016245,2000-01-02,12.4508,41.669998,20000.0,211.649994
3,ITM00016245,2000-01-02,12.4508,41.669998,15000.0,214.850006
4,ITM00016245,2000-01-02,12.4508,41.669998,5000.0,212.649994
5,ITM00016245,2000-01-02,12.4508,41.669998,10000.0,217.050003
...,...,...,...,...,...,...
19705,AUM00011035,2000-02-01,16.3564,48.248600,2380.0,205.050003
19706,AUM00011035,2000-02-01,16.3564,48.248600,68500.0,267.450012
19707,AUM00011035,2000-02-01,16.3564,48.248600,13700.0,209.449997


# long period - multiple stations

In [13]:
t = '2000-01-01/2020-01-01'
li =  ['AUM00011035', 'UKM00003693', 'GMM00010238', 'ITM00016245']
lc = ['11035', '03693', '10238', '16245']

### CUON

In [14]:
out = request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:51:52,719 INFO Welcome to the CDS
2021-02-11 13:51:52,719 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-11 13:51:52,782 INFO Request is queued
2021-02-11 13:51:53,821 INFO Request is running
2021-02-11 13:53:08,062 INFO Request is completed
2021-02-11 13:53:08,063 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.comprehensive_upper_air.retrieve-1613047968.7274065-24250-11-8dde8592-33e9-44cf-a4fe-dae5f37e1d01.zip to download.zip (814.2M)
2021-02-11 13:54:02,303 INFO Download rate 15M/s   


Request took: 130.86335253715515 seconds
2021-02-11 13:54:03,555 - upperair.cdm | __init__ - DEBUG - [OPEN] ./download/dest_0-20000-0-10238_air_temperature.nc


2021-02-11 13:54:03,555 DEBUG [OPEN] ./download/dest_0-20000-0-10238_air_temperature.nc


2021-02-11 13:54:03,563 - upperair.cdm | to_dataframe - INFO - Reading Profile on slice(None, None, None)


2021-02-11 13:54:03,563 INFO Reading Profile on slice(None, None, None)


...

2021-02-11 13:55:14,319 INFO Reading Profile on slice(None, None, None)


2021-02-11 13:55:14,932 - upperair.cdm | to_dataframe - DEBUG - Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


2021-02-11 13:55:14,932 DEBUG Read variables: dict_keys(['lat', 'lon', 'obs', 'plev', 'report_id', 'station_id', 'ta', 'time', 'trajectory', 'trajectory_index', 'trajectory_label'])


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,52.820000,9.930000,0,1420.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
1,52.820000,9.930000,0,1680.0,10000014384,BERGEN,203.300003,2000-01-01 05:00:00,10000014384
2,52.820000,9.930000,0,1850.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
3,52.820000,9.930000,0,2000.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
4,52.820000,9.930000,0,2430.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
...,...,...,...,...,...,...,...,...,...
2418888,48.248951,16.356871,0,100440.0,10000055272,WIEN/HOHE WARTE (11035-0),274.119995,2020-01-01 23:30:34,10000055272
2418889,48.248951,16.356871,0,100550.0,10000055272,WIEN/HOHE WARTE (11035-0),274.079987,2020-01-01 23:30:34,10000055272
2418890,48.248951,16.356871,0,100760.0,10000055272,WIEN/HOHE WARTE (11035-0),274.100006,2020-01-01 23:30:34,10000055272
2418891,48.248951,16.356871,0,100810.0,10000055272,WIEN/HOHE WARTE (11035-0),273.940002,2020-01-01 23:30:34,10000055272


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,52.820000,9.930000,0,1420.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
1,52.820000,9.930000,0,1680.0,10000014384,BERGEN,203.300003,2000-01-01 05:00:00,10000014384
2,52.820000,9.930000,0,1850.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
3,52.820000,9.930000,0,2000.0,10000014384,BERGEN,201.500000,2000-01-01 05:00:00,10000014384
4,52.820000,9.930000,0,2430.0,10000014384,BERGEN,201.899994,2000-01-01 05:00:00,10000014384
...,...,...,...,...,...,...,...,...,...
2418888,48.248951,16.356871,0,100440.0,10000055272,WIEN/HOHE WARTE (11035-0),274.119995,2020-01-01 23:30:34,10000055272
2418889,48.248951,16.356871,0,100550.0,10000055272,WIEN/HOHE WARTE (11035-0),274.079987,2020-01-01 23:30:34,10000055272
2418890,48.248951,16.356871,0,100760.0,10000055272,WIEN/HOHE WARTE (11035-0),274.100006,2020-01-01 23:30:34,10000055272
2418891,48.248951,16.356871,0,100810.0,10000055272,WIEN/HOHE WARTE (11035-0),273.940002,2020-01-01 23:30:34,10000055272


### IGRA 

In [15]:
out = request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)
display(out)
display(out.dropna(axis=0, how='any'))

2021-02-11 13:55:44,830 INFO Welcome to the CDS
2021-02-11 13:55:44,832 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-11 13:55:44,878 INFO Request is queued
2021-02-11 13:55:45,918 INFO Request is running
2021-02-11 14:04:03,691 INFO Request is completed
2021-02-11 14:04:03,694 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.insitu_reference.retrieve_test-1613048535.763101-25201-11-90298b62-e894-4c34-99ed-f58952a08f90.zip to download.zip (46.8M)
2021-02-11 14:04:05,100 INFO Download rate 33.3M/s  


Request took: 501.47721672058105 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
1,ITM00016245,2000-01-02,12.4508,41.669998,20000.0,211.649994
2,ITM00016245,2000-01-02,12.4508,41.669998,15000.0,214.850006
3,ITM00016245,2000-01-02,12.4508,41.669998,5000.0,212.649994
4,ITM00016245,2000-01-02,12.4508,41.669998,10000.0,217.050003
...,...,...,...,...,...,...
5746484,GMM00010238,2020-01-01,9.9247,52.815300,3910.0,204.850006
5746485,GMM00010238,2020-01-01,9.9247,52.815300,2500.0,200.649994
5746486,GMM00010238,2020-01-01,9.9247,52.815300,63000.0,264.450012


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,ITM00016245,2000-01-02,12.4508,41.669998,70000.0,262.450012
1,ITM00016245,2000-01-02,12.4508,41.669998,20000.0,211.649994
2,ITM00016245,2000-01-02,12.4508,41.669998,15000.0,214.850006
3,ITM00016245,2000-01-02,12.4508,41.669998,5000.0,212.649994
4,ITM00016245,2000-01-02,12.4508,41.669998,10000.0,217.050003
...,...,...,...,...,...,...
5746484,GMM00010238,2020-01-01,9.9247,52.815300,3910.0,204.850006
5746485,GMM00010238,2020-01-01,9.9247,52.815300,2500.0,200.649994
5746486,GMM00010238,2020-01-01,9.9247,52.815300,63000.0,264.450012
